In [ ]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            print("DECODE_STACKED() FAILED")
            pass
        yield obj

In [ ]:
#I don't think this is still needed
for obj in decode_stacked(fc):
    ind = 0
    try: versions = list(obj['versions'].keys())
    except: continue
    try: release_dates = obj['time']
    except: continue
    info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
    for key in versions:
        try:
            info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
            if pkg in info[key]['dependencies']: ind = 1
        except: continue
    if ind :
        dependent[pkg].append({obj['name']: info})

In [ ]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][0])
        break

In [ ]:
#user-info
#    pji
#        version 1.0
#        date
#        
#        version 1.1
#        date
#    gifnoc
#        version
#        date
        

In [ ]:
#pkgs = ['htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']
pkgs = ['ajv-keywords']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append(obj['name'])

In [ ]:
for k in dependent.keys():
    for i in dependent[k]:
        print(i)
        #print(k, dependent[k][:20])
        #print("len = ", len(dependent[k]))
        #break

In [ ]:
#ajv-keywords
    #jsconscript-proxy
    #senasaikou-yeoman
    #webpack-egoist
#user-info
    #ajv-keywords
        

In [ ]:
for k in dependent.keys():
    print(dependent[k][:20])
    print("len = ", len(dependent[k]))
    break

In [79]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for pkg in pkgs:
    dl = []
    for r in dt_range:
        url = base_url+r[0]+':'+r[1]+'/'+pkg
        try: r = requests.get(url)
        except: print('Timeout '+ pkg+'\n')
        
        try:
            result = r.content
            try:
                rj = json.loads(result.decode('utf-8', errors='ignore'))
                rj.pop('package', None)
                dl = dl + [rj]
            except: print('DecodeError '+ pkg+'\n')
        except: print('BadURL '+ pkg+'\n')
    #ent = {pkg: dl}    
    ent = {'name':pkg, 'dl': dl}

    coll.insert_one(ent)
    print(ent)

1
{'dl': [{'end': '2015-03-08', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-15', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-22', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-29', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-05', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-12', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-19', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-26', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-03', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-10', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-17', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-24', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-31', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-07', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-14', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-21', 'downloads': 0, 'start': '2015-06-14'}

In [66]:
import math
total_list = []
start_date = []
end_date = []
download_list = []

for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    for k in ent:
        download_list.append(k['downloads'])
        start_date.append(k["start"])
#map, reduce, filter, lambda
    log_list = [math.log(x+1) for x in download_list]
    difference_list = []
    for i in range(1,len(log_list)-1):
        dif = log_list[i] - log_list[i-1]
        difference_list.append( (log_list[i] - log_list[i-1]))
        if dif > 7:
            print("Start date of spike for " + r['name'] +": " +start_date[i])
        if dif < -7:
            print("Start date of fall for " + r['name'] +": " +start_date[i])
    #print(difference_list)
#print(log_list)
#print(download_list)


Start date of spike: 2017-07-12
Start date of spike: 2018-02-21
Start date of spike: 2018-02-05
Start date of fall: 2018-05-12


In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl2"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for k in dependent.keys():
    for i in dependent[k]:
        dl = []
        for r in dt_range:
            url = base_url+r[0]+':'+r[1]+'/'+i
            try: r = requests.get(url)
            except: print('Timeout '+ i+'\n')
        
            try:
                result = r.content
                try:
                    rj = json.loads(result.decode('utf-8', errors='ignore'))
                    rj.pop('package', None)
                    dl = dl + [rj]
                except: print('DecodeError '+ i+'\n')
            except: print('BadURL '+ i+'\n')
        #ent = {pkg: dl}    
        ent = {'name':i, 'dl': dl}
        coll.insert_one(ent)
        print(ent)

1
Timeout jsonscript-proxy

BadURL jsonscript-proxy

Timeout jsonscript-proxy

BadURL jsonscript-proxy

{'name': 'jsonscript-proxy', '_id': ObjectId('5bdca85a3a92c9110931923b'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '

{'name': 'senasaikou-yeoman', '_id': ObjectId('5bdca8b13a92c9110931923c'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '20

{'name': 'webpack-egoist', '_id': ObjectId('5bdca9073a92c9110931923d'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-

{'name': 'alex-d.js', '_id': ObjectId('5bdca95e3a92c9110931923e'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07

{'name': 'advanced-image-loader', '_id': ObjectId('5bdca9b43a92c9110931923f'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start':

{'name': 'webpack-addons-ylvis', '_id': ObjectId('5bdcaa0a3a92c91109319240'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': 

{'name': 'primeng-custom', '_id': ObjectId('5bdcaa603a92c91109319241'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-

{'name': 'jicli', '_id': ObjectId('5bdcaab73a92c91109319242'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, 

{'name': '@lgeiger/electron-builder', '_id': ObjectId('5bdcab113a92c91109319243'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'sta

{'name': '@touched/map-tool-project', '_id': ObjectId('5bdcab6b3a92c91109319244'), 'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'sta

In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

for r in coll.find():
    temp = r['name']
    print(temp)

In [77]:
import math, time, requests, sys, pymongo, json, datetime

dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]
#scaling up -- for pkg in pkgs?
print("For package " + pkg + ":")
for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    flag = 0
    for k in ent:
        if k["downloads"] > 10000: #ADDITION--------------
            flag = 1
    if flag == 1:    
        for k in ent:
            download_list.append(k["downloads"])
            start_date.append(k["start"])
#map, reduce, filter, lambda
            
        log_list = [math.log(x+1) for x in download_list]
   # difference_list = []
    if flag == 1:
            for i in range(1,len(log_list)-1):
                dif = log_list[i] - log_list[i-1]
       # difference_list.append(dif)
                if dif > 3.5:
                    print("Start date of spike for " + r['name'] +": " +start_date[i])
                if dif < -2:
                    print("Start date of fall for " + r['name'] +": " +start_date[i])
                #if r['name'] == 'yoshi':
                 #   print(dif)


For package ajv-keywords:
Start date of spike for @webpack-contrib/schema-utils: 2018-04-02
Start date of spike for schema-utils: 2017-03-06
Start date of spike for schema-utils: 2017-05-01
Start date of spike for read-config-file: 2017-07-12
Start date of fall for yoshi: 2017-12-11
Start date of spike for yoshi: 2018-04-18
Start date of fall for yoshi: 2018-04-26
Start date of fall for yoshi: 2018-05-28
Start date of fall for yoshi: 2018-07-07


In [ ]:
#revert to previous version and gets the old format of the versions. Gets dependent list with version history
pkgs = ['ajv-keywords']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []   
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append({obj['name']: info})